In [1]:
# importing the dependencies
import pandas as pd
import numpy as np

In [2]:
# importing the dataset of additional training data
data = pd.read_csv(r"C:\\Users\\Edison New\\Desktop\\HWG Q1 Reporting 2025\\Datasetsets shared\\Filtered_training_data.csv")

C:\Users\Edison New\AppData\Local\Temp\ipykernel_24856\2005496496.py:2: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(r"C:\\Users\\Edison New\\Desktop\\HWG Q1 Reporting 2025\\Datasetsets shared\\Filtered_training_data.csv")


In [3]:
# pivot table to count the number of households attended training
# for each district, cohort, cycle, and type
data1 = data.pivot_table(
    index=['district_name', 'cohort', 'cycle', 'type'],
    values='household_id',
    aggfunc='count'
).rename(columns={'household_id': 'No_of_HH_attended_training'})
data1= data1.reset_index()

In [4]:
# creating a new column for the total population of each district
data1['Total_population']=np.where(data1['district_name']=="Kitagwenda",2014,
                                                  np.where(data1['district_name']=="Rakai",2087,np.nan))

In [5]:
# replacing the total population with NaN for rows where the type contains 'champions'
if data1['type'].str.contains('champions', case=False).any():
   data1['Total_population'] = np.where(data1['type'].str.contains('champions', case=False), np.nan, data1['Total_population'])

In [6]:
# importing the dataset of designed champions
champions_designed=pd.read_csv("C:\\Users\\Edison New\\Desktop\\HWG Q1 Reporting 2025\\Datasetsets shared\\designed champions_clean.csv")

In [7]:
# Grouping the designed champions by district and type and resetting the index
champions_designed=champions_designed.groupby(['district_name','champion_type'])['Total_champions'].sum().reset_index()
champions_designed

,district_name,champion_type,Total_champions
0,Kitagwenda,agric champion,103
1,Kitagwenda,coffee champion,103
2,Kitagwenda,village champion,26
3,Kitagwenda,wash champion,103
4,Rakai,agric champion,104
5,Rakai,coffee champion,104
6,Rakai,village champion,17
7,Rakai,wash champion,104


In [8]:
# updating the total population in data1 based on the designed champions
# Looping through each row in the champions_designed DataFrame
for _, row in champions_designed.iterrows():
    district = row['district_name']
    champion_type = row['champion_type']
    total_champions = row['Total_champions']

    if champion_type == "agric champion":
        data1['Total_population'] = np.where(
            (data1['district_name'] == district) & (data1['type'] == "CATs / Agriculture Champions"),
            total_champions,
            data1['Total_population']
        )
    elif champion_type == "coffee champion":
        data1['Total_population'] = np.where(
            (data1['district_name'] == district) & (data1['type'] == "Coffee Champions - ToT 1"),
            total_champions,
            data1['Total_population']
        )
    elif champion_type == "wash champion":
        data1['Total_population'] = np.where(
            (data1['district_name'] == district) & (data1['type'] == "WASH Champions Training"),
            total_champions,
            data1['Total_population']
        )
    elif champion_type == "village champion":
        data1['Total_population'] = np.where(
            (data1['district_name'] == district) & (data1['type'] == "Village Champions"),
            total_champions,
            data1['Total_population']
        )

In [9]:
# creating a new column for the percentage attendance in data1 for trainings
data1['percentage_attendance']=data1['No_of_HH_attended_training']/data1['Total_population']

In [10]:
# having a look at the data1 dataframe
data1

,district_name,cohort,cycle,type,No_of_HH_attended_training,Total_population,percentage_attendance
0,Kitagwenda,2025,A,CATs / Agriculture Champions,126,103.0,1.223301
1,Kitagwenda,2025,A,Coffee Champions - ToT 1,104,103.0,1.009709
2,Kitagwenda,2025,A,HHT 1 - Day 2 - VSLAs,1940,2014.0,0.963257
3,Kitagwenda,2025,A,HHT 1 - Mindset and behavioral change,1933,2014.0,0.959782
4,Kitagwenda,2025,A,HHT 2 - Financial Literacy and VSLA and and Bu...,1928,2014.0,0.957299
5,Kitagwenda,2025,A,HOR 1,1908,2014.0,0.947368
6,Kitagwenda,2025,A,Village Champions,26,26.0,1.000000
7,Kitagwenda,2025,A,WASH - Training 1,1937,2014.0,0.961768
8,Kitagwenda,2025,A,WASH Champions Training,98,103.0,0.951456
9,Rakai,2025,A,CATs / Agriculture Champions,109,104.0,1.048077


In [11]:
# importing the dataset of additional training data to be used for the final output
filepath="C:\\Users\\Edison New\\Desktop\\HWG Q1 Reporting 2025\\Final outputs\\additional trainings.csv"
data1.to_csv(filepath, index=False)